In [13]:
import tqdm
import json
import requests
import os
from dotenv import load_dotenv 

load_dotenv()

BASE_URL = "https://api.the-odds-api.com/v4"
API_KEY = os.environ.get("ODDS_API_KEY")

REGIONS = ["eu"] # Betting regions
MARKETS = ["h2h"] # Betting markets to choose from 
SPORT = "basketball_nba_preseason" # Sport to interact with. Choose in the print sports cell.

THRESHOLD = 0.05 # # Accounts for percentage losses from vigorish

In [14]:
def get_sports(key) -> list:
    url = f"{BASE_URL}/sports/"

    response = requests.get(url, params= {
        "apiKey" : key
    })

    return list({sport["key"] for sport in response.json()})

def get_data(key, sport, regions, markets) -> list:

    url = f"{BASE_URL}/sports/{sport}/odds/"

    response = requests.get(url, params={
        "apiKey" : key,
        "regions" : ",".join(regions),
        "markets" : ",".join(markets),
        "oddsFormat" : "decimal" # American = +/-100 * (fraction odds of winning or losing, whichever >= 1). Sign based on win/loss

        # finding select bookmakers with frequent mispricings would improve arbitrage finding rates
        # "bookmakers" : TBA
    })

    print('Remaining requests', response.headers['x-requests-remaining'])
    print('Used requests', response.headers['x-requests-used'])
    
    return response.json()

In [15]:
def filter_arbitrages(data, threshold):
    arbitrages = {}
    for match in data:
        for bookmaker in match["bookmakers"]:
            for market in bookmaker["markets"]:
                implied_odds = sum(1/outcome["price"] for outcome in market["outcomes"])
                
                if implied_odds < 1 - threshold:

                    match_name = f'{match["home_team"]} vs. {match["away_team"]}'

                    if implied_odds > arbitrages.get(match_name, (-1, -1, -1))[-1]:
                        arbitrages[match_name] = {"bookmaker" : bookmaker["key"], "market" : market["key"], "odds" : implied_odds}

    return arbitrages

In [16]:
sports = get_sports(API_KEY)

print(sports) # See possible sports to choose from

['soccer_brazil_serie_b', 'soccer_belgium_first_div', 'soccer_turkey_super_league', 'basketball_euroleague', 'baseball_mlb_world_series_winner', 'soccer_england_league2', 'soccer_netherlands_eredivisie', 'americanfootball_cfl', 'icehockey_sweden_hockey_league', 'soccer_conmebol_copa_libertadores', 'soccer_germany_liga3', 'soccer_italy_serie_a', 'soccer_norway_eliteserien', 'soccer_france_ligue_one', 'golf_pga_championship_winner', 'soccer_switzerland_superleague', 'soccer_finland_veikkausliiga', 'americanfootball_ncaaf', 'americanfootball_nfl_super_bowl_winner', 'golf_masters_tournament_winner', 'soccer_uefa_champs_league', 'icehockey_nhl', 'soccer_uefa_europa_league', 'soccer_france_ligue_two', 'basketball_nba', 'soccer_japan_j_league', 'soccer_uefa_europa_conference_league', 'basketball_wnba', 'soccer_sweden_superettan', 'soccer_brazil_campeonato', 'soccer_england_league1', 'icehockey_nhl_championship_winner', 'golf_the_open_championship_winner', 'basketball_nba_preseason', 'rugbyuni

In [17]:
pricings_data = get_data(API_KEY, SPORT if SPORT else sports[0], REGIONS, MARKETS)

Remaining requests 477
Used requests 23


In [18]:
arbitrages = filter_arbitrages(pricings_data, THRESHOLD) 

In [19]:
if len(arbitrages) == 0:
    print(f"No arbitrages found for threshold {THRESHOLD} in sport {SPORT if SPORT else sports[0]}.")

for match_name in arbitrages:
    details = arbitrages[match_name]
    print(f'Match: {match_name}, Bookmaker: {details["bookmaker"]}, \
        Market: {details["market"]}, Total Implied Odds: {details["odds"]}')

Match: Philadelphia 76ers vs. Boston Celtics, Bookmaker: betfair_ex_eu,         Market: h2h_lay, Total Implied Odds: 0.9117693900302596
Match: Los Angeles Lakers vs. Sacramento Kings, Bookmaker: betfair_ex_eu,         Market: h2h_lay, Total Implied Odds: 0.9202195018995357
